# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import time 
import urllib
from scipy.stats import linregress
import scipy.stats as st


from api_keys import api_key

from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

#Range of lat and long
lat_range = (-90,90)
lng_range = (-180,180)

In [ ]:
## DATA ANALYSIS 

#1)

#2)

#3) 

## Generate Cities List

In [ ]:
# set up lists to hold reponse info
lat_lngs = []
cities = []


#create a set of random lat and long combos 
lats = np.random.uniform(lat_range[0], lat_range[1], size = 1400)
lngs = np.random.uniform(lng_range[0], lng_range[1], size = 1400)
lat_lngs = zip(lats,lngs)


# Loop through the list of cities and perform a request for data on each
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    if city not in cities: 
        cities.append(city)
        
len(cities)

In [ ]:
#Build URL
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial" 
settings = {"units": "imperial", "appid": api_key}
url = f"{url}appid={api_key}&units={units}"

In [ ]:
url

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

In [ ]:
# Create empty lists to append the API data to 
latitude = []
longitude = []
temp_max = []
humidity = []
cloud_coverage = []
wind_speed = []
city_name = []
country = []
datetime = []

# Start counter
record_counter = 1

#Print what curretly retrieving data for 
print(f"Start Data Retrieval Process")
print(f"-------------------------------")

#Loop through list of cities and append to empty lists 
for city in cities:  
    try: 
        response = requests.get(f"{url}&q={city}").json() 
        latitude.append(response["coord"]["lat"])
        longitude.append(response["coord"]["lon"])
        temp_max.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        cloud_coverage.append(response["clouds"]["all"])
        wind_speed.append(response["wind"]["speed"])
        city_name.append(response["name"])
        country.append(response["sys"]["country"])
        datetime.append(response["dt"])
        city_record = response["name"]
        print(f"Processing Record {record_counter} | {city_record}")
        print(f"{url}&q={city}")
        
        record_counter= record_counter + 1        
        
    # Pass if city not found
    except:
        print("City not found")
    continue

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# create a data frame from cities, lat, and temp
city_weather_dict = {
    "City": city_name,
    "Country": country,
    "Latitude": latitude,
    "Longitude": longitude,
    "Max Temperature": temp_max,
    "Humidity": humidity,
    "Cloud coverage": cloud_coverage,
    "Wind Speed": wind_speed,
    "City datetime": datetime
}
#create a data frame from the dictionary 
city_weather_df = pd.DataFrame(city_weather_dict)

city_weather_df.count()


In [ ]:
#a pandas dataframe is created and saved to a .csv from the data retrievied from the API 

city_weather_df.to_csv('city_weather_df.csv', encoding='utf-8', index=False)
city_weather_df


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humidity= city_weather_df["Humidity"].max()
humidity

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
city_weather_df.plot.scatter(x="Latitude", y= "Max Temperature",s=25, color = "blue", figsize=(12,7))
plt.xlabel('Latitude')
plt.ylabel('Max Temperature (F)')
plt.title('City Latitude vs. Max Temperature (03/29/21)')
plt.savefig("../WeatherPy/Weather_Images/Latitude_Temperature.png")

## Latitude vs. Humidity Plot

In [ ]:
city_weather_df.plot.scatter(x="Latitude", y= "Humidity",s=15, color = "blue",figsize=(12,7))
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('City Latitude vs. Humidity (03/29/21)')
plt.savefig("../WeatherPy/Weather_Images/Latitude_Humidity.png")


## Latitude vs. Cloudiness Plot

In [ ]:
city_weather_df.plot.scatter(x="Latitude", y= "Cloud coverage",s=25, color = "blue",figsize=(12,7))
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('City Latitude vs. Cloudiness (03/29/21)')
plt.savefig("../WeatherPy/Weather_Images/Latitude_Cloudiness.png");

## Latitude vs. Wind Speed Plot

In [ ]:
city_weather_df.plot.scatter(x="Latitude", y= "Wind Speed", marker = 'o',s=25, color = "blue",figsize=(12,7))
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('City Latitude vs. Wind Speed (mph) (03/29/21)')
plt.savefig("../WeatherPy/Weather_Images/Latitude_Wind Speed.png");

In [ ]:
hem=city_weather_df.sort_values(["Latitude"], ascending=True)
south = hem.head(177)
north=hem.tail(359)

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
north_temp = north.iloc[:,4]
lat_plot = north.iloc[:,2]
correlation = st.pearsonr(north_temp,lat_plot)


(slope, intercept, rvalue, pvalue, stderr) = st.linregress(north_temp, lat_plot)
regress_values = north_temp * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(north_temp,lat_plot)
plt.plot(north_temp,regress_values,"r-")
plt.annotate(line_eq,(0,20),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Maximum Temperature (F)')
plt.title("Northern Hemisphere Max Temp vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/North_Temperature.png")

print(f"The correlation coefficient for the northern hemisphere between max temp and latitude is {round(correlation[0],2)}")

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
south_temp = south.iloc[:,4]
lat_plot = south.iloc[:,2]
correlation = st.pearsonr(south_temp,lat_plot)

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(south_temp, lat_plot)
regress_values = south_temp * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(south_temp,lat_plot)
plt.plot(south_temp,regress_values,"r-")
plt.annotate(line_eq,(75,-50),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Maximum Temperature (F)')
plt.title("Southern Hemisphere Max Temp vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/South_Temperature.png")


print(f"The correlation coefficient for the southern hempisphere between max temp and latitude is {round(correlation[0],2)}")

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
north_humidity = north.iloc[:,5]
lat_plot = north.iloc[:,2]
correlation = st.pearsonr(north_humidity,lat_plot)

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(north_humidity, lat_plot)
regress_values = north_humidity * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(north_humidity,lat_plot)
plt.plot(north_humidity,regress_values,"r-")
plt.annotate(line_eq,(15,70),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title("Northern Hemisphere Humidity vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/North_Humidity.png")

print(f"The correlation coefficient for the northern hemisphere between humidity and latitude is {round(correlation[0],2)}")

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
south_humidity = south.iloc[:,5]
lat_plot = south.iloc[:,2]
correlation = st.pearsonr(south_humidity,lat_plot)

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(south_humidity, lat_plot)
regress_values = south_humidity * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(south_humidity,lat_plot)
plt.plot(south_humidity,regress_values,"r-")
plt.annotate(line_eq,(20,-50),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title("Southern Hemisphere Humidity vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/South_Humidity.png")


print(f"The correlation coefficient for the southern hemisphere between humidity and latitude is {round(correlation[0],2)}")

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
north_cloud = north.iloc[:,6]
lat_plot = north.iloc[:,2]
correlation = st.pearsonr(north_cloud,lat_plot)

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(north_cloud, lat_plot)
regress_values = north_cloud * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(north_cloud,lat_plot)
plt.plot(north_cloud,regress_values,"r-")
plt.annotate(line_eq,(45,75),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title("Northern Hemisphere Cloudiness vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/North_Cloud.png")


print(f"The correlation coefficient for the northern hemisphere between cloud coverage and latitude is {round(correlation[0],2)}")

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
south_cloud = south.iloc[:,6]
lat_plot = south.iloc[:,2]
correlation = st.pearsonr(south_cloud,lat_plot)

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(south_cloud, lat_plot)
regress_values = south_cloud * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(south_cloud,lat_plot)
plt.plot(south_cloud,regress_values,"r-")
plt.annotate(line_eq,(40,-50),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title("Southern Hemisphere Cloudiness vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/South_Cloud.png")


print(f"The correlation coefficient for the southern hemisphere between cloud coverage and latitude is {round(correlation[0],2)}")

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
north_wind = north.iloc[:,7]
lat_plot = north.iloc[:,2]
correlation = st.pearsonr(north_wind,lat_plot)

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(north_wind, lat_plot)
regress_values = north_wind * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(north_wind,lat_plot)
plt.plot(north_wind,regress_values,"r-")
plt.annotate(line_eq,(25,10),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title("Northern Wind Speed vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/North_Wind.png")


print(f"The correlation coefficient for the northern hemisphere between wind speed and latitude is {round(correlation[0],2)}")

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
south_wind = south.iloc[:,7]
lat_plot = south.iloc[:,2]
correlation = st.pearsonr(south_wind,lat_plot)

(slope, intercept, rvalue, pvalue, stderr) = st.linregress(south_wind, lat_plot)
regress_values = south_wind * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(south_wind,lat_plot)
plt.plot(south_wind,regress_values,"r-")
plt.annotate(line_eq,(15,-50),fontsize=15,color="red")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title("Southern Wind Speed vs. Latitude Linear Regression")
plt.show()
plt.savefig("../WeatherPy/Weather_Images/South_Wind.png")


print(f"The correlation coefficient for the southern hemisphere between wind speed and latitude is {round(correlation[0],2)}")